# Metrics for User Engagement with Search

Number and percentage of queries with a dwell time of more than 10 seconds. We're aiming to answer the question of whether users are finding full text search results relevant/useful.

We'll start by aggregating this on a daily basis using data from the SearchSatisfaction schema. We want to be able to filter by platform (desktop, mobile, android, iOS), bot vs. non-bot, and project, language edition, and if possible the language used in the search.

Looks like browser language headers are not stored in either the SearchSatisfaction nor the CirrusSearchRequest schema in the Data Lake. In other words, we'll either need to start storing that, or store data about user preferences on language (e.g. interface language) in those schemas.

What does this look like?

1. Identify all search sessions with first full-text SERP on a given day. We'll look one hour prior to the day starting and ignore those that had SERPs in that window.
2. Grab checkin events for those search sessions, up until 1 hour after midnight the day we're looking at.
3. Get the maximum checkin time for each session.
4. Aggregate sessions by maximum session time. All sessions that didn't have a checkin gets a checkin time of 0.
5. Store that data in a table.

We can then create a specific metric (number of sessions with dwell time of *x*) and create survival curves at will.

## Notes ##

The partition statement works reasonably well for daily aggregations. If we're doing hourly aggregations, we might want to restructure it so that the amount of data it sifts through is much smaller.

Should we calculate the funnel? In other words, number of sessions w/full text search, number of sessions with a click, maximum checkin time for that session. That would allow us to understand not only session length, but also whether users click on events. It'll require two separate tables, because we don't want to duplicate the number of sessions and click counts for each checkin length. We might just want to do that too.

Timestamps: we'll coalesce `dt` and `meta.dt`, and trust whatever comes out of it. When I investigated timestamps for searches on Commons I found that there are peaks around the various hour intervals, but they're incredibly small compared to the correct timestamp. To begin with, it's easier to trust these timestamps than develop heuristics to change them.

Event logging in SearchSatisfaction is only done on the desktop platform. This means that for these metrics, `platform` will always be `desktop`. We'll still have that column in the dataset to standarize on `project_family`, `wiki_db`, and `platform` as the key categories to filter on across both SearchSatisfaction and CirrusSearch.

We want to be able to separate activity by whether the searches are done by a bot, a user, or someone who's likely a bot. We'll adopt the `agent_type` field name used in Wikistats and reuse most of their categories. If the user agent defines the agent as a bot, we'll label it `bot`. If more than 50 searches are done in a session, it's likely a non-human agent and we'll label that `automated`. Otherwise, we'll label it `user`.

We define the sessions counts as `INT`, which allows for up to 2 billion search sessions per wiki per day. That should give us room to grow a bit. When we get to it, we can upgrade to `BIGINT`.

In [1]:
import datetime as dt

import pandas as pd
import numpy as np

from wmfdata import spark

## Configuring Timestamps

We'll call the day we're gathering data for `data_day`. We're also expecting this notebook to be run the day after, which we'll call `next_day`. In order to ignore search sessions that started on the previous day, we also define that day. Lastly, we set a limit of one hour after midnight UTC as the cutoff for data. In other words, we expect search sessions to be completed within one hour.

In [ ]:
next_day = dt.datetime.now(dt.timezone.utc).date()

data_day = next_day - dt.timedelta(days = 1)
previous_day = data_day - dt.timedelta(days = 1)

limit_timestamp = dt.datetime.combine(next_day, dt.time(hour = 1))

## Aggregation Tables

We define a set of tables in the Data Lake for aggregation of results.

In [17]:
search_session_funnel_table = 'search_dashboard_data.fulltext_funnel_counts'

search_checkin_table = 'search_dashboard_data.fulltext_checkin_counts'

In [ ]:
create_funnel_table_query = '''
CREATE TABLE {table_name} (
    project_family STRING COMMENT "project family (e.g. 'Wikipedia')",
    wiki_db STRING COMMENT "database name of the wiki (e.g. 'enwiki')",
    language_code STRING COMMENT "the language code of the wiki (e.g. 'en' for English, 'ar' for Arabic)",
    platform STRING COMMENT "desktop, mobile, Android, or iOS",
    agent_type STRING COMMENT "user, automated, or bot",
    log_date DATE COMMENT "the date of the aggregated search counts",
    num_fulltext_sessions INT COMMENT "number of search sessions with full text searches",
    num_click_sessions INT COMMENT "number of sessions with a click on a page in the results",
    num_checkin_sessions INT COMMENT "number of sessions with at least one checkin event",
    num_click_and_checkin_sessions INT COMMENT "number of sessions with both click and checkin events"
)
'''

Notes:

* I don't think `session_length` is a good name for a column, because we're not measuring how long a search session is, we're measuring how long the user spent on any given page they clicked on. I've decided to use `max_checkin` as the naming instead.
* I'll use `num_sessions` instead of `session_count` so that the naming is consistent with the funnel table.


In [29]:
create_checkin_table_query = '''
CREATE TABLE {table_name} (
    project_family STRING COMMENT "project family (e.g. 'Wikipedia')",
    wiki_db STRING COMMENT "database name of the wiki (e.g. 'enwiki')",
    language_code STRING COMMENT "the language code of the wiki (e.g. 'en' for English, 'ar' for Arabic)",
    platform STRING COMMENT "desktop, mobile, Android, or iOS",
    agent_type STRING COMMENT "user, automated, or bot",
    log_date DATE COMMENT "the date of the aggregated search counts",
    max_checkin INT COMMENT "maximum time in seconds for checkin events in a given session",
    num_sessions INT COMMENT "number of sessions with this maximum checkin"
)
'''

## Helper Functions

In [3]:
def make_partition_statement(start_ts, end_ts, prefix = ''):
    '''
    This takes the two timestamps and creates a statement that selects
    partitions based on `year`, `month`, and `day` in order to make our
    data gathering not use excessive amounts of data. It assumes that
    `start_ts` and `end_ts` are not more than a month apart, which should
    be a reasonable expectation for this notebook.
    
    An optional prefix can be set to enable selecting partitions for
    multiple tables with different aliases.
    
    :param start_ts: start timestamp
    :type start_ts: datetime.datetime
    
    :param end_ts: end timestamp
    :type end_ts: datetime.datetime
    
    :param prefix: prefix to use in front of partition clauses, "." is added automatically
    :type prefix: str
    '''
    
    if prefix:
        prefix = f'{prefix}.' # adds "." after the prefix
    
    # there are three cases:
    # 1: month and year are the same, output a "BETWEEN" statement with the days
    # 2: months differ, but the years are the same.
    # 3: years differ too.
    # Case #2 and #3 can be combined, because it doesn't really matter
    # if the years are the same in the month-selection or not.
    
    if start_ts.year == end_ts.year and start_ts.month == end_ts.month:
        return(f'''{prefix}year = {start_ts.year}
AND {prefix}month = {start_ts.month}
AND {prefix}day BETWEEN {start_ts.day} AND {end_ts.day}''')
    else:
        return(f'''
(
    ({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})
 OR ({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})
)''')

## Number of searches



In [4]:
## We count the overall number of searches in each session to later label
## those with >= 50 searches as "probably non-human"

## Q: How do we identify all platforms?
## A: We don't, SearchSatisfaction is only instrumented on desktop.

session_count_query = '''
WITH fulltext_sessions AS ( -- all search sessions started during the day of interest with >= 1 fulltext search
    SELECT
        event.searchsessionid AS session_id,
        FIRST_VALUE(wiki) AS wiki_db,
        FIRST_VALUE(useragent.is_bot) AS is_bot,
        COUNT(1) AS num_searches,
        COUNT(IF(event.source = "fulltext", 1, NULL)) AS num_fulltext_searches,
        MIN(coalesce(client_dt, meta.dt)) AS session_start_dt
    FROM event.searchsatisfaction AS ess
    WHERE {ess_partition_statement}
    AND event.subTest IS NULL
    AND event.action = "searchResultPage"
    AND event.isforced IS NULL -- only include non-test users
    GROUP BY event.searchsessionid
    HAVING
        TO_DATE(session_start_dt) = "{today}"
        AND num_fulltext_searches > 0
),
click_sessions ( -- sessions that had a click on a page or a visit page event
    SELECT
        event.searchsessionid AS session_id,
        1 AS clicked,
        MIN(coalesce(client_dt, meta.dt)) AS first_click_dt
    FROM fulltext_sessions AS fs
    INNER JOIN event.searchsatisfaction AS ess
    ON fs.session_id = ess.event.searchsessionid
    WHERE {ess_partition_statement}
    AND event.action IN ("click", "visitPage")
    AND coalesce(client_dt, meta.dt) < "{limit_timestamp}" -- until end of our data window
    AND coalesce(client_dt, meta.dt) > fs.session_start_dt
    GROUP BY event.searchsessionid
),
checkin_sessions ( -- sessions that had a checkin
    SELECT
        event.searchsessionid AS session_id,
        1 AS checked_in
    FROM fulltext_sessions AS fs
    INNER JOIN event.searchsatisfaction AS ess
    ON fs.session_id = ess.event.searchsessionid
    WHERE {ess_partition_statement}
    AND event.action = "checkin"
    AND coalesce(client_dt, meta.dt) < "{limit_timestamp}" -- until end of our data window
    AND coalesce(client_dt, meta.dt) > fs.session_start_dt
    GROUP BY event.searchsessionid
),
click_and_checkin_sessions ( -- sessions that had a click then a checkin
    SELECT
        event.searchsessionid AS session_id,
        1 AS clicked_and_checked_in
    FROM click_sessions AS cs
    INNER JOIN event.searchsatisfaction AS ess
    ON cs.session_id = ess.event.searchsessionid
    WHERE {ess_partition_statement}
    AND event.action = "checkin"
    AND coalesce(client_dt, meta.dt) < "{limit_timestamp}" -- until end of our data window
    AND coalesce(client_dt, meta.dt) > cs.first_click_dt
    GROUP BY event.searchsessionid
),
aggregated_sessions AS (
    SELECT
        fs.wiki_db,
        "desktop" AS platform,
        CASE
            WHEN fs.is_bot = true
            THEN "bot"
            WHEN fs.num_searches >= 50
            THEN "automated"
            ELSE "user"
        END AS agent_type,
        TO_DATE(fs.session_start_dt) AS log_date,
        COUNT(1) AS num_fulltext_sessions,
        COUNT(cs.clicked) AS num_click_sessions,
        COUNT(chs.checked_in) AS num_checkin_sessions,
        COUNT(cacs.clicked_and_checked_in) AS num_click_and_checkin_sessions
    FROM fulltext_sessions AS fs
    LEFT JOIN click_sessions AS cs
    ON fs.session_id = cs.session_id
    LEFT JOIN checkin_sessions AS chs
    ON fs.session_id = chs.session_id
    LEFT JOIN click_and_checkin_sessions AS cacs
    ON fs.session_id = cacs.session_id
    GROUP BY wiki_db,
        "desktop",
        CASE
            WHEN fs.is_bot = true
            THEN "bot"
            WHEN fs.num_searches >= 50
            THEN "automated"
            ELSE "user"
        END,
        TO_DATE(fs.session_start_dt)
),
wikis AS (
    SELECT
        database_code AS wiki_db,
        database_group AS project_family,
        language_code
    FROM canonical_data.wikis
)
INSERT INTO {aggregate_table}
SELECT
    wikis.project_family,
    aggs.wiki_db,
    wikis.language_code,
    aggs.platform,
    aggs.agent_type,
    aggs.log_date,
    aggs.num_fulltext_sessions,
    aggs.num_click_sessions,
    aggs.num_checkin_sessions,
    aggs.num_click_and_checkin_sessions
FROM aggregated_sessions AS aggs
JOIN wikis
ON aggs.wiki_db = wikis.wiki_db
'''

## Dwell Time Aggregation

We reuse much of the previous query to get search sessions. In this case, we make the query identify the maximum checkin time in the session, and then count sessions by said maximum. Sessions that did not have a checkin gets their time set to 0.

In [34]:
## We use the same approach as before to count the number of searches in a session
## and label those with more than 50 searches as "automated"

checkin_length_query = '''
WITH fulltext_sessions AS ( -- all search sessions started during the day of interest with >= 1 fulltext search
    SELECT
        event.searchsessionid AS session_id,
        FIRST_VALUE(wiki) AS wiki_db,
        FIRST_VALUE(useragent.is_bot) AS is_bot,
        COUNT(1) AS num_searches,
        COUNT(IF(event.source = "fulltext", 1, NULL)) AS num_fulltext_searches,
        MIN(coalesce(client_dt, meta.dt)) AS session_start_dt
    FROM event.searchsatisfaction AS ess
    WHERE {ess_partition_statement}
    AND event.subTest IS NULL
    AND event.action = "searchResultPage"
    AND event.isforced IS NULL -- only include non-test users
    GROUP BY event.searchsessionid
    HAVING
        TO_DATE(session_start_dt) = "{today}"
        AND num_fulltext_searches > 0
),
checkin_sessions ( -- sessions that had a checkin
    SELECT
        event.searchsessionid AS session_id,
        MAX(event.checkin) AS max_checkin
    FROM fulltext_sessions AS fs
    INNER JOIN event.searchsatisfaction AS ess
    ON fs.session_id = ess.event.searchsessionid
    WHERE {ess_partition_statement}
    AND event.action = "checkin"
    AND coalesce(client_dt, meta.dt) < "{limit_timestamp}" -- until end of our data window
    AND coalesce(client_dt, meta.dt) > fs.session_start_dt
    GROUP BY event.searchsessionid
),
aggregated_sessions AS (
    SELECT
        fs.wiki_db,
        "desktop" AS platform,
        CASE
            WHEN fs.is_bot = true
            THEN "bot"
            WHEN fs.num_searches >= 50
            THEN "automated"
            ELSE "user"
        END AS agent_type,
        TO_DATE(fs.session_start_dt) AS log_date,
        COALESCE(cs.max_checkin, 0) AS max_checkin,
        COUNT(1) AS num_sessions
    FROM fulltext_sessions AS fs
    LEFT JOIN checkin_sessions AS cs
    ON fs.session_id = cs.session_id
    GROUP BY wiki_db,
        "desktop",
        CASE
            WHEN fs.is_bot = true
            THEN "bot"
            WHEN fs.num_searches >= 50
            THEN "automated"
            ELSE "user"
        END,
        TO_DATE(fs.session_start_dt),
        COALESCE(cs.max_checkin, 0)
),
wikis AS (
    SELECT
        database_code AS wiki_db,
        database_group AS project_family,
        language_code
    FROM canonical_data.wikis
)
INSERT INTO {aggregate_table}
SELECT
    wikis.project_family,
    aggs.wiki_db,
    wikis.language_code,
    aggs.platform,
    aggs.agent_type,
    aggs.log_date,
    aggs.max_checkin,
    aggs.num_sessions
FROM aggregated_sessions AS aggs
JOIN wikis
ON aggs.wiki_db = wikis.wiki_db
'''

I've confirmed that the total number of sessions counted for 2021-07-12 for `nowiki` matches the number of search sessions counted in the overall session counts. I've also confirmed that the number of sessions with a checkin time `> 0` matches the `num_checkin_sessions` in the overall session count. This looks good to go to me.

## Data Gathering

Currently we've gathered about a month and a half of data to build a dashboard on for the team to play around with and gather feedback. If it's good to go, we can set up daily updates.

In [36]:
# set up days
first_day = dt.date(2021, 6, 2) # grab data from the first day of June 2021
last_day = dt.date(2021, 7, 13) # running on current day, so we have data until July 12

In [37]:
current_day = first_day

while current_day <= last_day:
    # calculate days
    next_day = current_day
    data_day = next_day - dt.timedelta(days = 1)
    previous_day = data_day - dt.timedelta(days = 1)

    limit_timestamp = dt.datetime.combine(next_day, dt.time(hour = 1))
    
    # print some helpful stuff
    print(f'running data gathering for {data_day} (simulating cron job on {current_day})')
    
    try:
        spark.run(checkin_length_query.format(
            today = data_day,
            limit_timestamp = limit_timestamp.isoformat(),
            ess_partition_statement = make_partition_statement(previous_day, next_day, prefix = 'ess'),
            ms_partition_statement = make_partition_statement(previous_day, next_day, prefix = 'ms'),
            aggregate_table = search_checkin_table
        ))
    except UnboundLocalError:
        # wmfdata currently (late Feb 2021) has an issue with DDL/DML SQL queries,
        # and so we ignore that error
        pass
    
    current_day += dt.timedelta(days = 1)

running data gathering for 2021-06-01 (simulating cron job on 2021-06-02)


PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


running data gathering for 2021-06-02 (simulating cron job on 2021-06-03)


PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


running data gathering for 2021-06-03 (simulating cron job on 2021-06-04)


...

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


running data gathering for 2021-07-11 (simulating cron job on 2021-07-12)


PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


running data gathering for 2021-07-12 (simulating cron job on 2021-07-13)


PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
